In [ ]:
from google.colab import drive
drive.mount('')

In [ ]:
!pip install datasets transformers
!pip install seqeval
! pip install -U datasets evaluate

In [ ]:
from datasets import DatasetDict, Dataset, Features, Sequence, Value, ClassLabel
from collections import defaultdict
from pathlib import Path

In [ ]:
def leer_archivo_bio(archivo_bio):
    """Lee un archivo .bio y devuelve un diccionario con tokens y etiquetas."""
    datos = defaultdict(list)
    with open(archivo_bio, 'r', encoding='utf-8') as f:
        lineas = f.readlines()

    tokens = []
    labels = []
    for num_linea, linea in enumerate(lineas, start=1):
        linea = linea.strip()
        if linea:
            partes = linea.split()
            if len(partes) != 2:
                raise ValueError(f"Error en línea {num_linea}: '{linea}'. Se esperaban 2 elementos.")
            palabra, etiqueta = partes
            tokens.append(palabra)
            labels.append(etiqueta)
        else:
            if tokens and labels:
                datos["tokens"].append(tokens)
                datos["ner_tags"].append(labels)
                tokens = []
                labels = []

    if tokens and labels:
        datos["tokens"].append(tokens)
        datos["ner_tags"].append(labels)

    return datos


def cargar_datasets_bio(rutas_archivos):
    """Carga archivos .bio y devuelve un DatasetDict."""
    datasets = {}
    for nombre, ruta in rutas_archivos.items():
        datos = leer_archivo_bio(ruta)
        datasets[nombre] = Dataset.from_dict(datos)

    return DatasetDict(datasets)



In [ ]:
# PASO 1: Despues de cargar los datos, primero se detecta todas las etiquetas únicas
def detectar_etiquetas_unicas(rutas_archivos):
    """Detecta automáticamente todas las etiquetas únicas en los archivos."""
    todas_etiquetas = set()

    for ruta in rutas_archivos.values():
        with open(ruta, 'r', encoding='utf-8') as f:
            for linea in f:
                linea = linea.strip()
                if linea:
                    partes = linea.split()
                    if len(partes) == 2:
                        _, etiqueta = partes
                        todas_etiquetas.add(etiqueta)

    # Ordenamos las etiquetas para que 'O' sea la última
    etiquetas_ordenadas = sorted(todas_etiquetas - {'O'}) + ['O']
    return etiquetas_ordenadas

In [ ]:
# Se definen los nombres de las rutas (paths) de los archivos .bio
rutas_archivos = {
    "train": "training.bio",
    "test": "testing_cleaned.bio",
    "valid": "validation_cleaned.bio"
}


In [ ]:

# Detectar automáticamente todas las etiquetas
LABELS = detectar_etiquetas_unicas(rutas_archivos)
print("Etiquetas detectadas:", LABELS)

# Cargar los datasets
dataset_dict = cargar_datasets_bio(rutas_archivos)

# Definir la estructura de features con las etiquetas detectadas
features = Features({
    "tokens": Sequence(Value("string")),
    "ner_tags": Sequence(ClassLabel(names=LABELS))
})

# Aplicar el casting a cada split
for split in dataset_dict:
    dataset_dict[split] = dataset_dict[split].cast(features)

# Mostrar información del dataset
print("\nDataset cargado correctamente:")
print(dataset_dict)

# Mostrar un ejemplo del conjunto de entrenamiento
print("\nEjemplo del train:")
print(dataset_dict["train"][0])

# Mostrar las características del dataset
print("\nCaracterísticas del dataset:")
print(dataset_dict["train"].features)

In [ ]:
dataset_dict

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-multilingual-uncased")

def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(
        examples["tokens"],
        truncation=True,
        is_split_into_words=True
    )

    labels = []
    for i, label in enumerate(examples["ner_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            if word_idx is None:
                label_ids.append(-100)
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])
            else:
                label_ids.append(-100)
            previous_word_idx = word_idx
        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

tokenized_datasets = dataset_dict.map(
    tokenize_and_align_labels,
    batched=True
)

In [ ]:
task = "ner" # Should be one of "ner", "pos" or "chunk"
model_checkpoint = "bert-base-multilingual-uncased"
batch_size = 8

In [ ]:
label_list = dataset_dict["train"].features[f"{task}_tags"].feature.names
label_list

In [ ]:
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer

model = AutoModelForTokenClassification.from_pretrained(model_checkpoint, num_labels=len(label_list))

In [ ]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer)

In [ ]:
model_multilingual_bert = model_checkpoint.split("/")[-1]
args = TrainingArguments(
    f"{model_multilingual_bert}-finetuned-{task}-prostata",
    eval_strategy = "epoch", # Changed from evaluation_strategy to eval_strategy
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=10,
    weight_decay=0.01,
    push_to_hub=True,
    hub_token=""
)

In [ ]:
try:
    from datasets import load_metric  # Para versiones antiguas
    metric = load_metric("seqeval")
except ImportError:
    from evaluate import load  # Para versiones nuevas
    metric = load("seqeval")

In [ ]:
import numpy as np

def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    # Remove ignored index (special tokens)
    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

Validación (eval_dataset): Se usa durante el entrenamiento para:

Ajustar hiperparámetros

Detener el entrenamiento temprano (early stopping)

Monitorizar el progreso

In [ ]:
trainer = Trainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["valid"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [ ]:
trainer.train()

Buenas prácticas:

No uses test para tomar decisiones: Solo para la evaluación final

Usa validación para ajustes: Early stopping, learning rate, etc.

Guarda test para el final: Como si fuera datos "reales" que el modelo nunca ha visto

In [ ]:
test_metrics = trainer.evaluate(tokenized_datasets["test"])
print("\n" + "="*50)
print(f"Resultados finales en conjunto de test:")
print(f"F1-score: {test_metrics['eval_f1']:.3f}")
print(f"Precisión: {test_metrics['eval_precision']:.3f}")
print(f"Recall: {test_metrics['eval_recall']:.3f}")
print("="*50)

In [ ]:
trainer.push_to_hub()

In [ ]:
label_names =  dataset_dict["train"].features["ner_tags"].feature.names
label_names

In [ ]:
predictions, labels, _ = trainer.predict(tokenized_datasets["test"])
predictions = np.argmax(predictions, axis=2)

# Remove ignored index (special tokens)
true_predictions = [
    [label_names[p] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]
true_labels = [
    [label_names[l] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]

results = metric.compute(predictions=true_predictions, references=true_labels)
results

SUBIR MODELO A HUGGIN FACE HUB

In [ ]:
# Instalar librerías
!pip install -q transformers huggingface_hub

# Autenticación
from huggingface_hub import login
login("....")  #TOKEN

# Guardar modelo y tokenizer (asegúrate de que la carpeta existe)
model.save_pretrained("")
tokenizer.save_pretrained("")

# Crear repositorio y subir archivos
from huggingface_hub import HfApi
api = HfApi()

# 1. Crear el repositorio (si no existe)
api.create_repo(
    repo_id="",
    repo_type="model",
    exist_ok=True
)

# 2. Subir archivos
api.upload_folder(
    folder_path="",
    repo_id="",
    repo_type="model",
    commit_message="Modelo basado en Multilingual BERT para NER en cáncer de próstata"
)

# 3. READ.ME
from huggingface_hub import ModelCard

card_content = """
---
license: apache-2.0
tags:
- medical
- ner
- spanish
- oncology
---

# Bert_base_multilingual_uncased_ner_Prostata_bio

Modelo Multilingual BERT entrenado para reconocimiento de entidades nombradas (NER) en textos médicos sobre cáncer de próstata.
"""

card = ModelCard(card_content)
card.push_to_hub("")

In [ ]:
#trainer.push_to_hub()